# Get data from the BigPEmu Debug Build

In [8]:
import struct
def recv_msg(sock):
    # Bigpemu has a 16 bytes header, with the data length
    # at positions 4:6
    header = recvall(sock, 16)
    if not header:
        return None
    raw_msglen = header[4:6]
    if not raw_msglen:
        return None
    msglen = struct.unpack('<H', raw_msglen)[0]
    # Read the message data
    return (header, msglen, recvall(sock, msglen))

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data

def get_response(sock, num_responses):
    # There's a predefined number of responses for each command,
    # so we have to specify how many to look for.
    resp = []
    for i in range(0,num_responses):
        data = recv_msg(sock)
        if not data:
            break
        resp += [data]
    return resp

## Launch BigPEmu and connect to the debug server

In [1]:
import socket
import os
import time


os.system("wine ./BigPEmu_v118-DEV/BigPEmuDev.exe ../tempest2k/t2000.abs &")
time.sleep(5)


In [3]:
import socket
import os
import time

# Launch bigpemu and wait a moment.
os.system("wine ./BigPEmu_v118-DEV/BigPEmuDev.exe ../tempest2k/t2000.abs &")
time.sleep(5)

HOST = "127.0.0.1"  # The server's hostname or IP address
PORT = 44123  # The port used by the server

# Connect to the debug server
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))



## Get some basic system info

In [5]:
hello  = b"\x01\x00\x00\x00\x08\x00\x00\x00\x14\x00\x00\x00" \
         b"\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00"

s.sendall(hello)
response = get_response(s,5)


## Remove all breakpoints

In [ ]:
"\x11\x00\x00\x00\x08\x00\x00\x00\x27\x00\x00\x00\x00\x00\x00\x00" \
"\x00\x00\x00\x00\x00\x00\x00\x00"


## Set breakpoint

In [ ]:
"\x11\x00\x00\x00\x30\x00\x00\x00\x28\x00\x00\x00\x00\x00\x00\x00" \
"\x01\x00\x00\x00\x00\x00\x00\x0000\xff\xff\xff\xff\xff\xff\xff\xff" \
"\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"


##  Resume?

In [ ]:
"\x03\x00\x00\x00\x00\x00\x00\x00\xd5\x00\x00\x00\x00\x00\x00\x00"


##  Set two breakpoints

In [ ]:
"\x11\x00\x00\x00\x58\x00\x00\x00\x2b\x00\x00\x00\x00\x00\x00\x00" \
"\x02\x00\x00\x00\x00\x00\x00\x00\xee\xee\xee\xee\xee\xee\xee\xee" \
"\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
"\xff\xff\xff\xff\xff\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00" \
"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
"\x00\x00\x00\x00\x00\x00\x00\x00"


##  Remove breakpoint

In [ ]:
"\x11\x00\x00\x00\x08\x00\x00\x00\x29\x00\x00\x00\x00\x00\x00\x00" \
"\x00\x00\x00\x00\x00\x00\x00\x00"


## Get a disassembly

In [283]:
hello3 = b"\x0f\x00\x00\x00\x18\x00\x00\x00\x20\x00\x00\x00\x00\x00\x00\x00" \
b"\x6c\x36\xf0\x00\x00\x00\x00\x00\x6c\x36\xf0\x00\x00\x00\x00\x00" \
b"\xc2\x00\x00\x00\x00\x00\x00\x00"

s.sendall(hello3)
response = get_response(s,1)
#print(len(response),response)


## Get a memory hex dump

In [284]:
"""
Code starts 802000
"""
# The address to start at
from_addr = 0x00802000
from_addr = struct.pack('<I', from_addr)
# The number of bytes to fetch
dump_len = 0x0830
dump_len = struct.pack('<H', dump_len)

hex_dump = \
b"\x0c\x00\x00\x00\x10\x00\x00\x00\x15\x00\x00\x00\x00\x00\x00\x00" \
+ from_addr + \
b"\x00\x00\x00\x00" \
+ dump_len + \
b"\x00\x00\x00\x00\x00\x00"

s.sendall(hex_dump)
response = get_response(s,1)

hex_dump = response[0][2]
WIDTH=32
mem = '\n'.join([
    ','.join(['{:02X}'.format(x) for x in hex_dump[l:l+WIDTH]]) 
    for l in range(0,len(hex_dump),WIDTH)
])
print(mem)

15,00,00,00,00,00,00,00,00,C8,9A,00,00,00,00,00,30,08,00,00,00,00,00,00,45,6E,67,69,6E,65,20,4E
6F,69,73,65,20,31,20,20,20,20,20,20,00,01,01,AC,00,9A,CD,00,00,11,A0,00,00,9A,CD,02,00,11,9E,00
50,6C,61,79,65,72,20,53,68,6F,74,20,4E,6F,72,6D,61,6C,20,32,00,02,01,AC,00,9A,DE,A4,00,08,E8,00
00,9A,DE,A4,00,00,00,00,45,6E,67,69,6E,65,20,4E,6F,69,73,65,20,20,20,20,20,20,20,20,00,03,01,AC
00,9A,E2,90,00,33,78,00,00,9A,EE,9E,00,25,94,00,50,6C,61,79,65,72,20,44,65,61,74,68,20,20,20,20
20,20,20,20,00,04,00,D6,00,00,00,00,00,54,9A,00,00,00,00,00,00,00,00,00,50,6C,61,79,65,72,20,44
65,61,74,68,20,32,20,20,20,20,20,20,00,05,01,AC,00,00,00,00,00,24,58,00,00,00,00,00,00,00,00,00
50,6C,61,79,65,72,20,53,68,6F,74,20,4E,6F,72,6D,61,6C,20,20,00,06,01,AC,00,9B,16,0C,00,07,A4,00
00,9B,16,0C,00,00,00,00,50,6C,61,79,65,72,20,4A,75,6D,70,20,20,20,20,20,20,20,20,20,00,07,01,AC
00,9B,1D,B4,00,18,DE,00,00,9B,1D,B4,00,00,00,00,43,72,61,63,6B,6C,65,20,20,20,20,20,20,20,20,20
20,20,20,20,00,08,00,D6,00,9B,36,96,00,4